# Detector Beetle Field Trial - Data Analysis

In [1]:
import pandas as pd
import fnmatch
import os
import gpxpy
import re

## End Points

Data source is a CSV text file, **detector_beetles1.csv** with the following fields:

* **beetle_id** Code engraved on beetle's elytrum
* **frequency** frequency (MHz) of radio transmitter glued to beetle's pronotum
* **rel_site** release site; Asan [13.473904, 144.708537]; Yigo [13.531333, 144.872750]
* **Notes** tracking notes
* **lat2** latitude of final observation (decimal degrees)
* **lon2** longitude of final observation (decimal degrees)
* **t2** timestamp of final observation (ChST)
* **extended_track_bearing** direction of travel from the final observation point (direct observation or radio direction finding)
* **end_point_located** 
* **in_tree** TRUE if end point is in a tree; determined visually or by radio direction finding
* **breeding_site** TRUE if otjer CRB present
* **flight_test_date** date on which beetle was flight tested and measured (ChST)
* **Length** lengthe of elytra (mm)
* **Width** width across elytra (mm)
* **Weight** mass (g)

In [8]:
df_endpoints = pd.read_csv('detector_beetles1.csv')
df_endpoints.sort(columns=['frequency'], inplace=True) # Sort by frequency
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 1000)
df_endpoints

,beetle_id,frequency,rel_site,Notes,lat2,lon2,t2,extended_track_bearing,end_point_located,in_tree,breeding_site,flight_test_date,Sex,Length,Width,Weight
0,2968,148.641,Asan,weak signal coming from swamp across from Asan Park; swamp,13.473933,144.708633,2015-08-11 09:18,180.00,False,NaN,NaN,2015-08-10,f,21.17,16.47,2.641
1,2977,148.671,Asan,Found at base of coco tree; just off beach,13.474933,144.708750,2015-08-11,NaN,True,False,NaN,2015-08-10,f,21.83,16.99,3.622
2,2991,148.693,Asan,Lost west over hill,13.472575,144.707141,2015-08-12 19:58:50,270.00,False,NaN,NaN,2015-08-11,m,25.86,20.63,5.774
3,2981,148.703,Asan,Lost west over hill; no track,NaN,NaN,NaN,270.00,False,NaN,NaN,2015-08-10,f,24.94,18.96,5.206
4,2971,148.732,Yigo,top of coconut tree adjust GPS,13.535104,144.873882,2015-08-12 11:00,NaN,True,True,NaN,2015-08-10,m,22.36,17.55,3.087
5,2929,148.752,NaN,lost; no track,NaN,NaN,2015-08-10,NaN,False,NaN,NaN,2015-08-05,f,25.41,20.29,4.220
6,2985,148.764,Asan,out of range; towards road,13.473374,144.708416,2015-08-10 21:01,180.00,False,NaN,NaN,2015-08-10,f,22.80,18.07,4.017
7,2963,148.782,Yigo,under trailer in the dirt; no track,13.529393,144.870394,2015-08-12 12:36,NaN,True,False,NaN,2015-08-10,m,20.33,16.28,3.179
8,2952,148.792,Yigo,beetle in breadfruit tree; 2952?; wooded,13.530583,144.872400,2015-08-10 09:30,NaN,True,True,True,2015-08-05,f,24.07,18.67,4.229
9,2978,148.820,Yigo,faint signal in woods; no track,13.529850,144.872300,2015-08-11 20:58,196.44,False,NaN,NaN,2015-08-10,m,25.22,20.15,5.213


## Data Mine All GPX Files for Waypoints of Tracked Beetles

Parse every **gpx** file contained in the ~/Dropbox/detectorBeetles folder and subfolders and extract data from waypoints in which a radio transmitter frequency ['148.???', '164.???'] is part of the **comment**
field. Waypoints are simulated for release sites and appended to the dataframe.

In [42]:
wp_list = []
for root, dirnames, filenames in os.walk('..'):
    for filename in fnmatch.filter(filenames, '*.gpx'):
        filepath = os.path.join(root, filename)      
        gpx_file = open(filepath).read()   
        gpx = gpxpy.parse(gpx_file)
        for wp in gpx.waypoints:
            s = wp.comment
            if s:
                if '148.' in s or '164.' in s:
                    frequency = re.findall('\d+.?\d+', s)[0] # Extract frequency from comment string
                    wp_list.append({'frequency':frequency, 'comment':wp.comment, 'filepath':filepath, 'name':wp.name, 'time':wp.time,
                                   'latitude':wp.latitude, 'longitude':wp.longitude})
df_waypoints = pd.DataFrame(wp_list)
df_waypoints.time = df_waypoints.time + pd.Timedelta(hours=10) # Add 10 h to convert from UTC to ChST

# Add waypoints for release sites.

simulated_waypoints = []
for i, row in df_waypoints.drop_duplicates('frequency').iterrows():
    if row.latitude > 13.5:
        rel_lat = 13.531333
        rel_lon = 144.872750
    else:
        rel_lat = 13.473904
        rel_lon = 144.708537
    simulated_waypoint = {'comment':'release point', 'frequency':row.frequency, 'latitude':rel_lat, 
                          'longitude':rel_lon, 'time':'2015-08-01 00:00:00'}
    simulated_waypoints.append(simulated_waypoint)
df_waypoints = df_waypoints.append(simulated_waypoints)
df_waypoints = df_waypoints[['frequency', 'comment', 'filepath', 'name', 'latitude', 'longitude', 'time']] # Reorder columns
df_waypoints.sort(columns=['frequency', 'time'], inplace=True) # Sort by frequency, time
pd.set_option('display.max_rows', 100)
df_waypoints

,frequency,comment,filepath,name,latitude,longitude,time
11,148.671,release point,NaN,NaN,13.473904,144.708537,2015-08-01 00:00:00
21,148.671,148.671 Under tree,../gpx by device by day/crb003/Waypoints_12-AUG-15.gpx,144,13.474914,144.708741,2015-08-12 16:31:51
29,148.671,148.671 No beetle just trans,../gpx by device by day/crb003/Waypoints_13-AUG-15.gpx,152,13.474930,144.708739,2015-08-13 10:00:17
12,148.693,release point,NaN,NaN,13.473904,144.708537,2015-08-01 00:00:00
24,148.693,148.693 Up hill,../gpx by device by day/crb003/Waypoints_12-AUG-15.gpx,147,13.472575,144.707141,2015-08-12 19:58:50
10,148.732,release point,NaN,NaN,13.531333,144.872750,2015-08-01 00:00:00
33,148.732,148.732 Over fense,../gpx by device by day/crb003/Waypoints_11-AUG-15.gpx,139,13.533517,144.871667,2015-08-11 20:10:48
20,148.732,148.732 Top of coconut tree adjust,../gpx by device by day/crb003/Waypoints_12-AUG-15.gpx,143,13.534695,144.873841,2015-08-12 10:57:45
7,148.764,release point,NaN,NaN,13.473904,144.708537,2015-08-01 00:00:00
17,148.764,148.764 Out of range towards road,../gpx by device by day/crb003/Waypoints_10-AUG-15.gpx,Uuuu,13.473374,144.708416,2015-08-10 21:01:26


In [43]:
s = 'frequency|track\n'
for freq in df_waypoints.frequency.unique():
    s += '{}|LINESTRING('.format(freq)
    df = df_waypoints[df_waypoints.frequency==freq]
    for i, row in df.iterrows():
        s += '{} {},'.format(row.longitude, row.latitude)
    s = s[:-1] + ')\n'
print(s) 
with open("track2.csv", "w") as text_file:
    text_file.write(s)

frequency|track
148.671|LINESTRING(144.708537 13.473904,144.708741 13.474914,144.708739 13.47493)
148.693|LINESTRING(144.708537 13.473904,144.707141 13.472575)
148.732|LINESTRING(144.87275 13.531333,144.871667 13.533517,144.873841 13.534695)
148.764|LINESTRING(144.708537 13.473904,144.708415959 13.4733739961)
148.792|LINESTRING(144.708537 13.473904,144.707267 13.472583)
148.851|LINESTRING(144.87275 13.531333,144.873406 13.532034,144.87348 13.532032,144.8734 13.532036)
148.873|LINESTRING(144.87275 13.531333,144.87086 13.532304,144.870622 13.532258,144.707886 13.475064,144.70897 13.4747)
148.932|LINESTRING(144.708537 13.473904,144.775814 13.492996,144.775814 13.492996)
148.992|LINESTRING(144.708537 13.473904,144.707217 13.473563,144.708077 13.473998,144.70812 13.474004,144.708144 13.473946)
164.132|LINESTRING(144.87275 13.531333,144.872727 13.53132,144.871666 13.532509)
164.157|LINESTRING(144.87275 13.531333,144.708445 13.474407,144.707195 13.472294,144.706364 13.471207,144.871812 13.532

In [44]:
import pyproj
import math
import random

wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
utm55n=pyproj.Proj("+init=EPSG:32655") # UTM coords, zone 55N, WGS84 datum

def extended_track(longitude, latitude, distance, bearing, bearing_jitter=0):
    """ Returns a track extended from a point for a given distance in a given direction.
    longitude, latitude: starting point in WGS84
    distance: distance to extend track in meters
    angle: angle in compass degrees (east = 90)
    angle_jitter: range of a uniform random number to be added or subracted from angle;
        for eaxmple, if angle=90 and angle_jitter=0.5, track will have an angle between 89.5 and 90.5
        
    Returns a track as a LINESTRING
    """
    x, y = pyproj.transform(wgs84, utm55n, longitude, latitude)
    if bearing_jitter > 0:
        bearing = random.uniform(bearing-bearing_jitter, bearing+bearing_jitter)
    angle = math.radians(90.0 - bearing) # angle in radians
    x_final, y_final = (x + distance * math.cos(angle), y + distance * math.sin(angle))
    longitude_final, latitude_final = pyproj.transform(utm55n, wgs84, x_final, y_final) # Convert back to lnn/lat
    return longitude_final, latitude_final

#extended_track(144.70901, 13.47140, 500, 90, 0.5)

def get_bearing(lat1, lon1, lat2, lon2):
    """ Returns bearing between two geographical coordinates in decimal degrees
    """
    startLat = math.radians(lat1)
    startLong = math.radians(lon1)
    endLat = math.radians(lat2)
    endLong = math.radians(lon2)
    dLong = endLong - startLong
    dPhi = math.log(math.tan(endLat/2.0+math.pi/4.0)/math.tan(startLat/2.0+math.pi/4.0))
    if abs(dLong) > math.pi:
         if dLong > 0.0:
             dLong = -(2.0 * math.pi - dLong)
         else:
             dLong = (2.0 * math.pi + dLong)
    bearing = (math.degrees(math.atan2(dLong, dPhi)) + 360.0) % 360.0;
    return bearing

#get_bearing(43.682213, -70.450696, 43.682194, -70.450769)
# 250.20613449

In [45]:
import re

def parse_linestring(linestring):
    """ Returns coordinates as a list of tuples
    """
    coord_list = []
    for line in s.split(','):
        coord = re.findall(r'[+-]?[0-9.]+', line)
        coord = tuple(coord)
        coord_list.append(coord)
    return coord_list

s = 'LINESTRING(144.708741 13.473904, 144.708537 13.473904)'    
parse_linestring(s)

[('144.708741', '13.473904'), ('144.708537', '13.473904')]

In [46]:
import simplekml

kml = simplekml.Kml()

track_style = simplekml.Style()
track_style.linestyle.color = simplekml.Color.yellow
track_style.linestyle.width = 4

extended_track_style = simplekml.Style()
extended_track_style.linestyle.color = simplekml.Color.orange
extended_track_style.linestyle.width = 2

# Tracks

for i, row in df_endpoints.iterrows():
    if row.rel_site == 'Asan':
        coord1 = (144.708537, 13.473904)
    elif row.rel_site == 'Yigo':
        coord1 = (144.87275, 13.531333)
    else:
        continue

    if not pd.isnull(row.lon2) and not pd.isnull(row.lat2):
        coord2 = (row.lon2, row.lat2)
    else:
        coord2 = coord1
        
    ls = kml.newlinestring(name=str(row.beetle_id), coords=[coord1, coord2])
    ls.style = track_style
    
    if not pd.isnull(row.extended_track_bearing):
        extended_coord = extended_track(coord2[0], coord2[1], 500, row.extended_track_bearing, 5.0)
        ls = kml.newlinestring(coords=[coord2, extended_coord])
        ls.style = extended_track_style  
        
for i, row in df_endpoints.iterrows():
    if not pd.isnull(row.lon2) and not pd.isnull(row.lat2):
        pnt = kml.newpoint(name=str(row.beetle_id), coords=[(row.lon2, row.lat2)])      

kml.save('tracks.kml')

In [47]:
df_endpoints

,beetle_id,frequency,rel_site,Notes,lat2,lon2,d2,t2,extended_track_bearing,end_point_located,in_tree,breeding_site
0,2968,148.641,Asan,weak signal coming from swamp across from Asan Park; swamp,13.473933,144.708633,11-Aug-15,9:18,180.00,False,NaN,NaN
1,2977,148.671,Asan,Found at base of coco tree; just off beach,13.474933,144.708750,11-Aug-15,NaN,NaN,True,False,NaN
2,2991,148.693,Asan,Lost west over hill,13.472575,144.707141,2015-08-12 19:58:50,NaN,270.00,False,NaN,NaN
3,2981,148.703,Asan,Lost west over hill; no track,NaN,NaN,NaN,NaN,270.00,False,NaN,NaN
4,2971,148.732,Yigo,top of coconut tree adjust GPS,13.535104,144.873882,12-Aug-15,11:00,NaN,True,True,NaN
5,2929,148.752,NaN,lost; no track,NaN,NaN,10-Aug-15,NaN,NaN,False,NaN,NaN
6,2985,148.764,Asan,out of range; towards road,13.473374,144.708416,08/10/15,09:01:00 PM,180.00,False,NaN,NaN
7,2963,148.782,Yigo,under trailer in the dirt; no track,13.529393,144.870394,12-Aug-15,12:36,NaN,True,False,NaN
8,2952,148.792,Yigo,beetle in breadfruit tree; 2952?; wooded,13.530583,144.872400,10-Aug-15,9:30,NaN,True,True,True
9,2978,148.820,Yigo,faint signal in woods; no track,13.529850,144.872300,11-Aug-15,20:58,196.44,False,NaN,NaN


## Add Morphometrics to Data Set

In [54]:
df_morphometrics = pd.read_excel('../flight-tested-beetles.xls')
merged_left = pd.merge(left=df_endpoints,right=df_morphometrics, how='left', left_on='beetle_id', right_on='Beetle #')
merged_left.drop(['Beetle #','Outcome','EW','%EW','Area'], axis=1, inplace=True)
merged_left.tail(5)
merged_left.to_csv('merged.csv', index=False)

In [53]:
merged_left


,beetle_id,frequency,rel_site,Notes,lat2,lon2,d2,t2,extended_track_bearing,end_point_located,in_tree,breeding_site,flight_test_date,Sex,Length,Width,Weight
0,2968,148.641,Asan,weak signal coming from swamp across from Asan Park; swamp,13.473933,144.708633,11-Aug-15,9:18,180.00,False,NaN,NaN,2015-08-10,f,21.17,16.47,2.641
1,2977,148.671,Asan,Found at base of coco tree; just off beach,13.474933,144.708750,11-Aug-15,NaN,NaN,True,False,NaN,2015-08-10,f,21.83,16.99,3.622
2,2991,148.693,Asan,Lost west over hill,13.472575,144.707141,2015-08-12 19:58:50,NaN,270.00,False,NaN,NaN,2015-08-11,m,25.86,20.63,5.774
3,2981,148.703,Asan,Lost west over hill; no track,NaN,NaN,NaN,NaN,270.00,False,NaN,NaN,2015-08-10,f,24.94,18.96,5.206
4,2971,148.732,Yigo,top of coconut tree adjust GPS,13.535104,144.873882,12-Aug-15,11:00,NaN,True,True,NaN,2015-08-10,m,22.36,17.55,3.087
5,2929,148.752,NaN,lost; no track,NaN,NaN,10-Aug-15,NaN,NaN,False,NaN,NaN,2015-08-05,f,25.41,20.29,4.220
6,2985,148.764,Asan,out of range; towards road,13.473374,144.708416,08/10/15,09:01:00 PM,180.00,False,NaN,NaN,2015-08-10,f,22.80,18.07,4.017
7,2963,148.782,Yigo,under trailer in the dirt; no track,13.529393,144.870394,12-Aug-15,12:36,NaN,True,False,NaN,2015-08-10,m,20.33,16.28,3.179
8,2952,148.792,Yigo,beetle in breadfruit tree; 2952?; wooded,13.530583,144.872400,10-Aug-15,9:30,NaN,True,True,True,2015-08-05,f,24.07,18.67,4.229
9,2978,148.820,Yigo,faint signal in woods; no track,13.529850,144.872300,11-Aug-15,20:58,196.44,False,NaN,NaN,2015-08-10,m,25.22,20.15,5.213
